<a href="https://colab.research.google.com/github/inesCherif/cuda-vs-cpu-loop-comparison/blob/main/tpCuda.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# TP : Comparaison entre une boucle CPU et une boucle GPU avec CUDA

## 🎯 Objectif :
- Se familiariser avec l'exécution de programmes CUDA dans Google Colab.
- Comparer une implémentation séquentielle (CPU) et une implémentation parallèle (GPU) pour une opération simple : incrémenter les éléments d’un tableau.

---

## 🧠 Introduction

Lorsqu'on travaille avec de grands volumes de données, l'exécution séquentielle sur le processeur (CPU) peut devenir lente. CUDA (Compute Unified Device Architecture) permet d'exécuter des programmes sur la carte graphique (GPU), en exploitant **le parallélisme massif** du GPU.

Dans ce TP, on compare :

- Un programme classique en **C**, utilisant une boucle `for` sur CPU.
- Une version **CUDA**, utilisant des threads pour faire le même travail **en parallèle** sur GPU.

L'opération effectuée : **incrémenter chaque élément d’un tableau de 5 entiers.**


# **Configuration de l'envirennement CUDA sur google colab**

In [ ]:
!apt-get update
!apt-get install cuda-11-2

Get:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,381 kB]
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [2,775 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,812 kB]
Get:13 http://archive.ubuntu.com/ubunt

In [ ]:
!nvcc --version
!nvidia-smi

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0
Fri Apr  4 18:48:24 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8       

In [ ]:
%%writefile vector_add.cu
#include <iostream>
#include <cuda_runtime.h>

#define N 5  // Nombre d'éléments

__global__ void addVectors(int *a, int *b, int *c) {
    int index = threadIdx.x;
    c[index] = a[index] + b[index];
}

int main() {
    int h_a[N] = {1, 2, 3, 4, 5};
    int h_b[N] = {10, 20, 30, 40, 50};
    int h_c[N] = {0};

    int *d_a, *d_b, *d_c;

    // Allocation de la mémoire sur le GPU
    cudaMalloc((void**)&d_a, N * sizeof(int));
    cudaMalloc((void**)&d_b, N * sizeof(int));
    cudaMalloc((void**)&d_c, N * sizeof(int));

    // Copier les tableaux d'entrée du host vers le device
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(d_b, h_b, N * sizeof(int), cudaMemcpyHostToDevice);

    // Lancer le noyau
    addVectors<<<1, N>>>(d_a, d_b, d_c);

    // Copier le résultat du device vers le host
    cudaMemcpy(h_c, d_c, N * sizeof(int), cudaMemcpyDeviceToHost);

    // Afficher les résultats
    std::cout << "Résultat de l'addition de vecteurs : ";
    for (int i = 0; i < N; i++) {
        std::cout << h_c[i] << " ";
    }
    std::cout << std::endl;

    // Libérer la mémoire sur le GPU
    cudaFree(d_a);
    cudaFree(d_b);
    cudaFree(d_c);

    return 0;
}

Overwriting vector_add.cu


In [ ]:
!nvcc -arch=sm_75 -o vector_add vector_add.cu

In [ ]:
!./vector_add

Résultat de l'addition de vecteurs : 11 22 33 44 55 


# **Réalisation TP**

## **🔹Version CPU (incrémentation séquentielle)**

In [ ]:
%%writefile increment_cpu.cpp
#include <iostream>
#define N 5

void increment_cpu(int *a, int n) {
    for (int i = 0; i < n; i++) {
        a[i] = a[i] + 1;
    }
}

int main() {
    int a[N] = {1, 2, 3, 4, 5};

    increment_cpu(a, N);

    std::cout << "Résultat CPU : ";
    for (int i = 0; i < N; i++) {
        std::cout << a[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}

Writing increment_cpu.cpp


### Compilation + Exécution (CPU)

In [ ]:
!g++ increment_cpu.cpp -o increment_cpu
!./increment_cpu

Résultat CPU : 2 3 4 5 6 


### 🔍 Explication du code CPU

- `increment_cpu` est une fonction qui prend un tableau `a` et l’incrémente élément par élément avec une boucle `for`.
- L’exécution se fait **de manière séquentielle** : un seul cœur de CPU traite tous les éléments un par un.
- Le tableau final affiché est : `2 3 4 5 6`

## 🔹**Version GPU (CUDA – traitement parallèle)**

In [ ]:
%%writefile increment_gpu.cu
#include <iostream>
#include <cuda_runtime.h>

#define N 5

__global__ void increment_gpu(int *a, int n) {
    int i = threadIdx.x;
    if (i < n) {
        a[i] = a[i] + 1;
    }
}

int main() {
    int h_a[N] = {1, 2, 3, 4, 5};
    int *d_a;

    cudaMalloc((void**)&d_a, N * sizeof(int));
    cudaMemcpy(d_a, h_a, N * sizeof(int), cudaMemcpyHostToDevice);

    increment_gpu<<<1, N>>>(d_a, N);

    cudaMemcpy(h_a, d_a, N * sizeof(int), cudaMemcpyDeviceToHost);

    std::cout << "Résultat GPU : ";
    for (int i = 0; i < N; i++) {
        std::cout << h_a[i] << " ";
    }
    std::cout << std::endl;

    cudaFree(d_a);
    return 0;
}

Writing increment_gpu.cu


### Compilation + Exécution (GPU)

In [ ]:
!nvcc -arch=sm_75 -o increment_gpu increment_gpu.cu
!./increment_gpu

Résultat GPU : 2 3 4 5 6 


### ⚡ Explication du code GPU

- La fonction `increment_gpu` est une fonction spéciale CUDA (noyau) marquée par `__global__`.
- Chaque **thread** prend un index `i` et incrémente un seul élément du tableau.
- On exécute `N` threads en parallèle : un pour chaque élément du tableau.
- Résultat affiché : `2 3 4 5 6`, identique à la version CPU, mais traitement fait **en parallèle**.


## ⚖️ **Comparaison entre CPU et GPU**

| Aspect               | Version CPU (C)           | Version GPU (CUDA)                      |
|----------------------|---------------------------|------------------------------------------|
| Type d’exécution     | Séquentielle              | Parallèle (multi-thread)                |
| Syntaxe              | `for` classique           | `__global__` avec `threadIdx.x`         |
| Résultat             | Identique                 | Identique                               |
| Rapidité             | Suffisante pour petits N  | Optimale pour grands N                  |
| Complexité du code   | Simple                    | Plus complexe (gestion mémoire CUDA)    |

## 🧩 Conclusion

Même si les deux codes donnent le **même résultat**, la version GPU est plus adaptée aux **opérations lourdes et massives**. Ce TP montre comment CUDA permet de paralléliser des tâches répétitives, en exploitant la puissance du GPU.